In [1]:
import pandas as pd 
import numpy as np 


In [ ]:
def donchian_breakout(df: pd.DataFrame,lookback: int):
    df['upper'] =df['close'].rolling(lookback - 1).max().shift
    df['lower'] =df['close'].rolling(lookback - 1).max().shift
    df['lower'] = np.nan
    df.loc[df['close'] > df['upper'],'signal'] = 1
    df.loc[df['close'] > df['lower'],'signal'] = -1
    df['signal'] = df['signal'].ffill()

def signal_returns(df: pd.Dataframe,fee_amt:float = 0.001):
    #input df assumed to have close and signal columns
    df['log_return'] = np.log(df['close']).diff().shift(-1) #next log return
    df['strategy_return'] = df['signal'] * df['log_return']

    position_change = df['signal'].diff().abs()
    df['strategy_return'] -= position_change * fee_amt
    df['equity_curve'] =df['strategy_return'].cumsum()

def perfomance_metrics(df: pd.DataFrame, n_bars_in_years: int):
    r = df['strategy_return']

    # compute perf metrics
    profit_factor = r[r>0].sum() / r[r < 0].abs().sum()
    sharpe = r.mean() / r.std()
    sortino = r.mean() / (r[r < 0].std())

    #annualize sharpe and metrics
    sharpe *= n_bars_in_years ** 0.5
    sortino *= n_bars_in_years ** 0.5

    return {"Profit Factor": profit_factor, "sharpe Ratio": sharpe, "sortino": sortino }
        